In [1]:
#pragma cling load("libcurl")
#include "./lib/everest.hpp"
#include <list>
using namespace TEMPLET;
using namespace std;

In [ ]:
{
    taskengine eng("enter your login", "enter your password");
    string token;
    eng.get_access_token(token);
    eng.free_access_token();
    cout <<"access token: "<< token << endl;
    eng.get_app_description("lyapunov");    
}

In [2]:
const int NUM_WORKERS = 1;
const int NUM_TASKS = 1;

double rho[NUM_TASKS]={28.0};
double sigma[NUM_TASKS]={10.0};
double beta[NUM_TASKS]={2.66666};

int cur;

In [3]:
struct worker;
list<worker*> idle_worker_list;

In [4]:
struct worker:task {
    /**/
	int pos_in_arr;
	/**/
	worker():task(){}
    
    void on_worker_ready(){
        // put the result
        /**/
        json& out=result();
        double L1 = out["L1"];
        double L2 = out["L2"];
        double L3 = out["L3"];
        
        cout << "for rho = " << rho[pos_in_arr] << " sigma = " << sigma[pos_in_arr] << " beta = " << beta[pos_in_arr] << endl;
        cout << "L1 = " << L1 << " L2 = " << L2 << " L3 = " << L3 << endl << endl;
        /**/
        
        idle_worker_list.push_back(this);
        run_tasks();
    }
    
    static void run_tasks(){
        // run tasks while having tasks and free workers
        while(!idle_worker_list.empty() && /**/cur < NUM_TASKS/**/){
            worker*w = idle_worker_list.front();
            idle_worker_list.pop_front();

            /**/
            json in;
            
            in["name"] = "Lyapunov-exp";
            
            in["inputs"]["r"] = rho[cur];
            in["inputs"]["s"] = sigma[cur];
            in["inputs"]["b"] = beta[cur];
            
            w->input(in);
            w->pos_in_arr = cur;
            cur++;
            /**/
            
            w->submit([=](){w->on_worker_ready();});
        }
    }
};

In [5]:
taskengine eng("enter access token (the cell #2 output) ");

worker worker_arr[NUM_WORKERS];
    
for(int i=0;i<NUM_WORKERS;i++){
    worker_arr[i].set_app_id("5ece24c22f00001f74656926");// this is lyapunov-exponent app id, enter yours if necessary
    worker_arr[i].set_engine(eng);
    idle_worker_list.push_back(&worker_arr[i]);
}

cur = 0;
worker::run_tasks(); // submitting the first portion of tasks
eng.wait_all();

for rho = 28 sigma = 10 beta = 2.66666
L1 = 0.880075 L2 = -0.002137 L3 = -14.5385

